In [1]:
from visdom import Visdom

class VisdomLinePlotter(object):
    """Plots to Visdom"""
    def __init__(self, env_name='main'):
        self.viz = Visdom()
        self.env = env_name
        self.plots = {}
    def plot(self, var_name, split_name, title_name, x, y):
        if var_name not in self.plots:
            self.plots[var_name] = self.viz.line(X=np.array([x,x]), Y=np.array([y,y]), env=self.env, opts=dict(
                legend=[split_name],
                title=title_name,
                xlabel='Epochs',
                ylabel=var_name
            ))
        else:
            self.viz.line(X=np.array([x]), Y=np.array([y]), env=self.env, win=self.plots[var_name], name=split_name, update = 'append')
            
            
    
vis = VisdomLinePlotter()

Setting up a new session...


In [2]:
from pytorch_transformers import AdamW, WarmupLinearSchedule

In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline

Using TensorFlow backend.


In [4]:
torch.cuda.is_available()

True

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla V100-PCIE-16GB'

In [6]:
import numpy as np
import math
import pandas as pd

import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import *

from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier

In [7]:
df = pd.read_csv('reddit_train.csv')
df2 = pd.read_csv('reddit_test.csv')
df = df.sample(1000, random_state=1).copy()
df2 = df2.sample(1000, random_state=1).copy()
df.head()

,id,comments,subreddits
6670,6670,Yeah but euron's about to bring cersei tyrion ...,gameofthrones
49567,49567,All of his videos are sarcastic and funny...hi...,conspiracy
50796,50796,I love those scenes but it wouldn't have made ...,movies
22310,22310,You do get a smidge of hp for every point of c...,wow
54037,54037,New MMORPG lets you play as someone playing a ...,wow


In [8]:
df['category_id'], mapping = df['subreddits'].factorize()

In [9]:
for i in range(df.comments.values.shape[0]):
    df.comments.values[i] = df.comments.values[i][:512]

In [10]:
# Create sentence and label lists
sentences = df.comments.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df.category_id.values

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)

In [12]:


tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['[CLS]', 'yeah', 'but', 'euro', '##n', "'", 's', 'about', 'to', 'bring', 'ce', '##rse', '##i', 'ty', '##rion', 'as', 'a', 'gift', '[SEP]']


In [13]:
# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 512

In [14]:
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [15]:
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [16]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [17]:
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.05)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.05)

In [18]:
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [19]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 2

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [20]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=20)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace

In [21]:

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta', 'Layer.Norm']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]



In [22]:
# This variable contains all of the hyperparemeter information our training loop needs
# optimizer = BertAdam(optimizer_grouped_parameters,
#                      lr=2e-5,
#                      warmup=.1)
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=1e-8)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=0, t_total=-1)

In [23]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4
step = 0
# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
    # Training

    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    
    # Train the data for one epoch
    for batch in tqdm(train_dataloader,desc='batch',leave=False):
        step = step + 1
#         if(step%10==0):
#             print('step (out of 8750): ' + str(step) + '\r')
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        train_loss_set.append(loss.item())  
        vis.plot('loss', 'train_loss', 'Loss',step,loss.item())
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        scheduler.step()


        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        if(step%500 == 0): # in [500, 1000, 3000, 6000]
            print('step (out of 8750): ' + str(step))
            print("Train loss: {}".format(tr_loss/nb_tr_steps))


            # Validation

            # Put model in evaluation mode to evaluate loss on the validation set
            model.eval()

            # Tracking variables 
            eval_loss, eval_accuracy = 0, 0
            nb_eval_steps, nb_eval_examples = 0, 0

            # Evaluate data for one epoch
            for batch in validation_dataloader:
                # Add batch to GPU
                batch = tuple(t.to(device) for t in batch)
                # Unpack the inputs from our dataloader
                b_input_ids, b_input_mask, b_labels = batch
                # Telling the model not to compute or store gradients, saving memory and speeding up validation
                with torch.no_grad():
                    # Forward pass, calculate logit predictions
                    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

                # Move logits and labels to CPU
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()

                tmp_eval_accuracy = flat_accuracy(logits, label_ids)

                eval_accuracy += tmp_eval_accuracy
                nb_eval_steps += 1

            print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
            vis.plot('accuracy', 'val_acc', 'val_acc',step,eval_accuracy/nb_eval_steps)
            torch.save(model, 'bert_large.pt')

batch:  29%|██▊       | 136/475 [00:45<02:05,  2.70it/s]


batch:  57%|█████▋    | 272/475 [01:37<01:19,  2.57it/s]


batch:  86%|████████▌ | 408/475 [02:31<00:24,  2.76it/s]


batch:   5%|▌         | 24/475 [00:09<02:48,  2.67it/s]

step (out of 8750): 500
Train loss: 3.2455451583862303
Validation Accuracy: 0.04



batch:   8%|▊         | 38/475 [00:29<02:48,  2.59it/s]

In [35]:
df2.comments.values.shape

(30000,)

In [36]:
# df2 = df2.iloc[:1000]

In [37]:
for i in range(df2.comments.values.shape[0]):
    df2.comments.values[i] = df2.comments.values[i][:512]

In [38]:
len(sentences)

1000

In [39]:
# Create sentence and label lists
sentences = df2.comments.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]

In [40]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 



In [41]:
input_ids.shape

(30000, 512)

In [42]:
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
# prediction_labels = torch.tensor(labels)
  
batch_size = 32


prediction_data = TensorDataset(prediction_inputs, prediction_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [43]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions = []

# Predict 
for batch in tqdm(prediction_dataloader,desc='batch',leave=False):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()

    # Store predictions and true labels
    predictions.append(logits)

In [44]:
main_preds = []
for i in range(len(predictions)):
    main_preds += list(np.argmax(predictions[i], axis=1))
    
print(len(main_preds))

30000


In [45]:
main_preds

[19,
 16,
 6,
 14,
 4,
 7,
 19,
 6,
 13,
 16,
 13,
 12,
 13,
 17,
 3,
 14,
 18,
 14,
 9,
 13,
 5,
 17,
 15,
 19,
 15,
 11,
 7,
 12,
 8,
 4,
 10,
 7,
 1,
 14,
 12,
 12,
 19,
 4,
 1,
 12,
 0,
 6,
 16,
 4,
 12,
 9,
 17,
 15,
 5,
 11,
 2,
 6,
 17,
 5,
 7,
 17,
 18,
 13,
 16,
 1,
 8,
 13,
 13,
 13,
 17,
 7,
 1,
 3,
 13,
 9,
 2,
 4,
 6,
 15,
 16,
 7,
 16,
 0,
 4,
 6,
 14,
 17,
 9,
 8,
 17,
 13,
 6,
 13,
 0,
 19,
 13,
 18,
 17,
 17,
 6,
 5,
 17,
 11,
 4,
 4,
 8,
 8,
 19,
 15,
 8,
 4,
 16,
 11,
 2,
 16,
 10,
 2,
 13,
 5,
 11,
 5,
 9,
 6,
 17,
 3,
 19,
 14,
 10,
 16,
 16,
 15,
 15,
 13,
 2,
 18,
 19,
 0,
 19,
 15,
 18,
 18,
 13,
 0,
 8,
 3,
 12,
 16,
 3,
 4,
 16,
 15,
 4,
 9,
 12,
 16,
 10,
 13,
 3,
 9,
 1,
 17,
 12,
 5,
 17,
 9,
 6,
 15,
 5,
 4,
 13,
 15,
 15,
 4,
 18,
 9,
 4,
 0,
 5,
 13,
 7,
 3,
 12,
 5,
 5,
 10,
 3,
 4,
 2,
 10,
 12,
 5,
 4,
 17,
 16,
 6,
 17,
 10,
 2,
 6,
 16,
 11,
 2,
 9,
 12,
 11,
 13,
 17,
 10,
 17,
 2,
 10,
 11,
 8,
 12,
 19,
 18,
 13,
 13,
 2,
 16,
 10,
 16,
 19,
 5,


In [46]:
test_preds = pd.DataFrame()
test_preds['Id'] = df2['id']
test_preds['Category'] = mapping[main_preds]

In [47]:
test_preds

,Id,Category
0,0,baseball
1,1,europe
2,2,anime
3,3,worldnews
4,4,funny
...,...,...
29995,29995,movies
29996,29996,movies
29997,29997,Overwatch
29998,29998,gameofthrones


In [48]:
test_preds.to_csv("test2.csv", index=False)
from IPython.display import FileLink, FileLinks
FileLink('test2.csv')

/network/home/penmetss/comp551/test2.csv

In [18]:
model.load_state_dict(a.state_dict())

<All keys matched successfully>

In [17]:
a = torch.load('main_model_v2.pt')